In [1]:
import os
import cv2
import csv
import glob
import json
import pprint

import math

import numpy as np
import pandas as pd

import scipy
import matplotlib.pyplot as plt

In [2]:
data_gt = pd.read_csv("/home/kunaldargan/ROBO_SURGERY/features.csv", index_col="VIDEO")

In [3]:
normalized_data_gt=(data_gt-data_gt.min())/(data_gt.max()-data_gt.min())

In [4]:
normalized_data_gt

,NUMBER_OF_HITS,HIT_INTENSITY,NUMBER_OF_TUG,TOTAL_FRAMES_IN_TUG,NUMBER_OF_JERK_FRAMES,TIME,NUMBER_OF_RINGDROP,VARIANCE,ARC_LENGTH,CURVATURE_MAX_COUNT
VIDEO,,,,,,,,,,
E1_t10,0.4,0.0,0.10,0.155405,0.000000,0.078603,0.0,0.340096,0.170420,0.026316
E1_t11,0.4,0.0,0.05,0.128378,0.000000,0.096798,0.0,0.275642,0.148542,0.131579
E1_t12,0.4,0.0,0.15,0.175676,0.000000,0.242358,0.0,0.184697,0.267084,0.263158
E1_t1,0.4,1.0,0.05,0.060811,0.272727,0.224163,0.0,0.261132,0.227164,0.236842
E1_t2,0.0,0.0,0.05,0.020270,0.000000,0.055313,1.0,0.181222,0.047672,0.184211
E1_t3,0.0,0.0,0.10,0.128378,0.136364,0.024017,0.0,0.392002,0.074573,0.078947
E1_t4,0.2,0.0,0.05,0.114865,0.227273,0.167394,0.0,0.380739,0.312127,0.184211
E1_t5,0.2,0.0,0.05,0.101351,0.136364,0.061135,0.0,0.414221,0.141920,0.105263
E1_t6,0.4,0.0,0.05,0.108108,0.000000,0.096798,0.0,0.434658,0.233527,0.105263


In [5]:
df_expert = pd.read_csv('/home/kunaldargan/ROBO_SURGERY/expert_scores_trainee_scores.csv',  index_col="Video")

In [6]:
df_expert

,Hand-eye Coordination,Instrument-Tissue Manipulation,Dexterity,Flow of procedure,Effectualness,Overall
Video,,,,,,
E1_t1,3,3,3,3,4,15
E1_t2,3,3,3,2,1,8
E1_t3,4,3,4,4,4,18
E1_t4,3,2,4,4,4,16
E1_t5,3,3,4,4,4,17
E1_t6,3,3,4,4,4,17
E1_t7,4,4,4,4,4,20
E1_t8,3,3,4,4,4,17
E1_t9,4,4,4,4,4,20


In [7]:
JOINT = normalized_data_gt.join(df_expert)

In [8]:
JOINT

,NUMBER_OF_HITS,HIT_INTENSITY,NUMBER_OF_TUG,TOTAL_FRAMES_IN_TUG,NUMBER_OF_JERK_FRAMES,TIME,NUMBER_OF_RINGDROP,VARIANCE,ARC_LENGTH,CURVATURE_MAX_COUNT,Hand-eye Coordination,Instrument-Tissue Manipulation,Dexterity,Flow of procedure,Effectualness,Overall
VIDEO,,,,,,,,,,,,,,,,
E1_t10,0.4,0.0,0.10,0.155405,0.000000,0.078603,0.0,0.340096,0.170420,0.026316,4,3,4,4,4,19
E1_t11,0.4,0.0,0.05,0.128378,0.000000,0.096798,0.0,0.275642,0.148542,0.131579,4,3,4,4,4,19
E1_t12,0.4,0.0,0.15,0.175676,0.000000,0.242358,0.0,0.184697,0.267084,0.263158,4,2,4,4,4,18
E1_t1,0.4,1.0,0.05,0.060811,0.272727,0.224163,0.0,0.261132,0.227164,0.236842,3,3,3,3,4,15
E1_t2,0.0,0.0,0.05,0.020270,0.000000,0.055313,1.0,0.181222,0.047672,0.184211,3,3,3,2,1,8
E1_t3,0.0,0.0,0.10,0.128378,0.136364,0.024017,0.0,0.392002,0.074573,0.078947,4,3,4,4,4,18
E1_t4,0.2,0.0,0.05,0.114865,0.227273,0.167394,0.0,0.380739,0.312127,0.184211,3,2,4,4,4,16
E1_t5,0.2,0.0,0.05,0.101351,0.136364,0.061135,0.0,0.414221,0.141920,0.105263,3,3,4,4,4,17
E1_t6,0.4,0.0,0.05,0.108108,0.000000,0.096798,0.0,0.434658,0.233527,0.105263,3,3,4,4,4,17


In [9]:
JOINT.columns

Index(['NUMBER_OF_HITS', 'HIT_INTENSITY', 'NUMBER_OF_TUG',
       'TOTAL_FRAMES_IN_TUG', 'NUMBER_OF_JERK_FRAMES', 'TIME',
       'NUMBER_OF_RINGDROP', 'VARIANCE', 'ARC_LENGTH', 'CURVATURE_MAX_COUNT',
       'Hand-eye Coordination', 'Instrument-Tissue Manipulation', 'Dexterity',
       'Flow of procedure', 'Effectualness', 'Overall'],
      dtype='object')

In [10]:
from sklearn.model_selection import cross_val_score
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

from sklearn.svm import SVC, SVR
from sklearn.gaussian_process import GaussianProcessClassifier, GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor , AdaBoostClassifier, AdaBoostRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import mean_squared_error, accuracy_score, r2_score

from sklearn.metrics import label_ranking_loss 

from sklearn.model_selection import GridSearchCV
import random
from scipy.stats import pearsonr

from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict


In [15]:
class Trainer:
    
    def __init__(self, df):
        
        self.X = df[['NUMBER_OF_HITS', 'HIT_INTENSITY', 'NUMBER_OF_TUG',
                  'TOTAL_FRAMES_IN_TUG', 'TIME','NUMBER_OF_JERK_FRAMES', 'NUMBER_OF_RINGDROP',
                   'VARIANCE', 'ARC_LENGTH', 'CURVATURE_MAX_COUNT']]
        self.data = df.copy() 
#         self.onenc = OneHotEncoder(handle_unknown='ignore')

    
    def run(self, classifiers,params, names, prediction_label=None,method="classify", split_size=0.3, k_folds = 3,digitize = False):
        
        results = []
        #"select feature type for y label"
        assert prediction_label is not None and prediction_label in self.data.columns

        Y = self.data[[prediction_label]]

        if  prediction_label == "Overall" and digitize: #for overall scores
            Y = np.digitize(np.array(Y),np.array([5.0, 10.0, 15.0, 20.0]))
            
        X_train, X_test, y_train, y_test = train_test_split(self.X, Y, test_size=split_size, random_state=random.randint(0, 500)) #70 | 30
#         sss = StratifiedShuffleSplit(1, test_size=0.5, random_state=random.randint(0, 500))

#         for train_index, test_index in sss.split(self.X, Y):
#             X_train, X_test = self.X.iloc[train_index], self.X.iloc[test_index]
#             y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
        
        gt_y_test = y_test[[prediction_label]].values.ravel()

        for parameters, model in zip(params, classifiers):
                print(prediction_label,model)
                if method == "classify":
                    clf = GridSearchCV(model, parameters, scoring="accuracy",cv=k_folds)#make_scorer(label_ranking_loss)
                    clf.fit(X_train, y_train.values.ravel())
#                     scores = cross_val_score(clf,X_train, y_train, cv=k_folds)
                    res = {}
                    test = clf.predict(X_test)
                    
                    res["estimator_type"] = model.__class__.__name__
                    res["accuracy_test"] = accuracy_score(test, gt_y_test)
                    target_names = ["bad","fair","good","excellent"]
                    print(classification_report(gt_y_test,test))
                    
                    test = cross_val_predict(clf.best_estimator_, X, Y.values.ravel(), cv=10)
                    r = pearsonr(test, Y.values.ravel())

                    res["correlation coefficient"] = r[0]
                    res["accuracy_cross_val"] = clf.best_score_
                    res["estimator"] = clf.best_estimator_
                    res["parameter_search_dict"] = clf.best_params_
                    
                    results.append(res)

                    
                elif method == "regression":
                    clf = GridSearchCV(model, parameters,scoring=make_scorer(mean_squared_error), cv=k_folds)
                    clf.fit(X_train, y_train.values.ravel())
#                     scores = cross_val_score(clf,X_train, y_train, cv=k_folds)
                    res = {}
                    test = clf.predict(X_test) #y_pred
                    test = test.ravel()
                    #test = [round(x) for x in test] #rounding to nearest integer
                    mse = mean_squared_error(test,gt_y_test) #ypred vs gt y for test data
                    
                    test = cross_val_predict(clf.best_estimator_, X, Y.values.ravel(), cv=10)

                    r = pearsonr(test, Y.values.ravel())
                    res["correlation coeff."] = r[0]

                    res["estimator_type"]= model.__class__.__name__
                    res["mse_test"] = mse
                    res["mse_cross_val"] = clf.best_score_
                    res["estimator"] = clf.best_estimator_
                    res["parameter_search_dict"] = clf.best_params_

                    results.append(res)
                    
                    
        return results


In [16]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM",
 "Decision Tree", "Random Forest"] # "Neural Net", "Gaussian Process", "AdaBoost", "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(),
    SVC(), #gamma=2, C=1
    SVC(), #kernel="linear", C=0.025
    DecisionTreeClassifier(),
    RandomForestClassifier()]
#     MLPClassifier(alpha=1, max_iter=1000),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     AdaBoostClassifier(),
#     GaussianNB(),
#     QuadraticDiscriminantAnalysis()]
params_classifiers = [
    {"n_neighbors":[3,4,5], "algorithm" :["auto", "ball_tree", "kd_tree", "brute"]},
    {'kernel': ['rbf'], 'gamma': [1,1e-1, 1e-2, 1e-3, 1e-4],'C': [1, 10, 100, 1000]}, #svc
    {'kernel': ['linear'], 'C': [0.025, 0.25, 1, 10, 100, 1000]}, #linear kernel svc
    {"max_depth": [4,5,6,7,8],"splitter":["best", "random"]}, #decision tree
    {"max_depth":[4,5,6,7,8], "n_estimators":[10,20], "max_features":[1,2,5]}, #Random forest
]

names_reg = [ "Nearest Neighbors Regressor", "Linear SVR Regressor", "RBF SVR",
         "Decision Tree Regressor", "Random Forest Regressor"]# "Neural Net Regressor",  "Gaussian Process Regressor", "AdaBoost Regressor"]


regressors = [
    KNeighborsRegressor(5),    
    SVR(),
    SVR(),
    DecisionTreeRegressor(),
    RandomForestRegressor()
#     MLPRegressor(alpha=1, max_iter=1000),
#     GaussianProcessRegressor(1.0 * RBF(1.0)),
#     AdaBoostRegressor(),
    
]

params_regressors = [
    {"n_neighbors":[3,4,5], "algorithm" :["auto", "ball_tree", "kd_tree", "brute"]},
    {'kernel': ['rbf'], 'gamma': [1,1e-1, 1e-2, 1e-3, 1e-4],'C': [1, 10, 100, 1000]}, #svc
    {'kernel': ['linear'], 'C': [0.025, 0.25, 1, 10, 100, 1000]}, #linear kernel svc
    {"max_depth": [4,5,6,7,8],"splitter":["best", "random"]}, #decision tree
    {"max_depth":[4,5,6,7,8], "n_estimators":[10,20], "max_features":[1,2,5]}, #Random forest
]
# classifiers = classifiers + regressors

In [17]:
trainer = Trainer(JOINT)

In [18]:
preds = ['Hand-eye Coordination', 'Instrument-Tissue Manipulation', 'Dexterity','Flow of procedure', 'Overall']#'Effectualness'

In [19]:
classifiers

[KNeighborsClassifier(),
 SVC(),
 SVC(),
 DecisionTreeClassifier(),
 RandomForestClassifier()]

In [20]:
regressors

[KNeighborsRegressor(),
 SVR(),
 SVR(),
 DecisionTreeRegressor(),
 RandomForestRegressor()]

In [21]:
from sklearn.linear_model import LinearRegression


In [22]:
observations = ['NUMBER_OF_HITS', 'HIT_INTENSITY', 'NUMBER_OF_TUG',
            'TOTAL_FRAMES_IN_TUG', 'TIME','NUMBER_OF_JERK_FRAMES', 'NUMBER_OF_RINGDROP',
            'VARIANCE', 'ARC_LENGTH', 'CURVATURE_MAX_COUNT']

In [23]:
prediction_labels = ['Hand-eye Coordination', 'Instrument-Tissue Manipulation', 'Dexterity','Flow of procedure', 'Effectualness', 'Overall']

In [24]:
X = JOINT[observations]

In [25]:
res_classifier = {}
for p in preds:
    res_classifier[p]=trainer.run(classifiers,params_classifiers, names,p)

Hand-eye Coordination KNeighborsClassifier()


/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


              precision    recall  f1-score   support

           1       0.33      1.00      0.50         1
           2       0.00      0.00      0.00         3
           3       0.38      0.30      0.33        10
           4       0.53      0.64      0.58        14

    accuracy                           0.46        28
   macro avg       0.31      0.49      0.35        28
weighted avg       0.41      0.46      0.43        28

Hand-eye Coordination SVC()
              precision    recall  f1-score   support

           1       0.33      1.00      0.50         1
           2       1.00      0.33      0.50         3
           3       0.20      0.20      0.20        10
           4       0.43      0.43      0.43        14

    accuracy                           0.36        28
   macro avg       0.49      0.49      0.41        28
weighted avg       0.40      0.36      0.36        28

Hand-eye Coordination SVC()


/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


              precision    recall  f1-score   support

           1       0.25      1.00      0.40         1
           2       0.00      0.00      0.00         3
           3       0.22      0.20      0.21        10
           4       0.47      0.50      0.48        14

    accuracy                           0.36        28
   macro avg       0.23      0.42      0.27        28
weighted avg       0.32      0.36      0.33        28

Hand-eye Coordination DecisionTreeClassifier()
              precision    recall  f1-score   support

           1       0.25      1.00      0.40         1
           2       1.00      0.33      0.50         3
           3       0.38      0.50      0.43        10
           4       0.70      0.50      0.58        14

    accuracy                           0.50        28
   macro avg       0.58      0.58      0.48        28
weighted avg       0.60      0.50      0.51        28

Hand-eye Coordination RandomForestClassifier()


...

              precision    recall  f1-score   support

           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         3
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         2
          16       0.33      0.33      0.33         3
          17       0.00      0.00      0.00         2
          18       0.00      0.00      0.00         6
          19       0.20      0.17      0.18         6
          20       0.12      0.33      0.18         3

    accuracy                           0.11        28
   macro avg       0.05      0.07      0.06        28
weighted avg       0.09      0.11      0.09        28

Overall DecisionTreeClassifier()


/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_split

              precision    recall  f1-score   support

           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         2
          11       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         3
          15       0.00      0.00      0.00         2
          16       0.00      0.00      0.00         3
          17       0.00      0.00      0.00         2
          18       0.50      0.33      0.40         6
          19       0.00      0.00      0.00         6
          20       0.17      0.67      0.27         3

    accuracy                           0.14        28
   macro avg       0.07      0.10      0.07        28
weighted avg       0.12      0.14      0.11        28

Overall RandomForestClassifier()


/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


              precision    recall  f1-score   support

           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         2
          11       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         3
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         2
          16       0.50      0.33      0.40         3
          17       0.00      0.00      0.00         2
          18       1.00      0.17      0.29         6
          19       0.00      0.00      0.00         6
          20       0.20      0.67      0.31         3

    accuracy                           0.14        28
   macro avg       0.15      0.11      0.09        28
weighted avg       0.29      0.14      0.14        28



In [26]:
res_classifier

{'Hand-eye Coordination': [{'estimator_type': 'KNeighborsClassifier',
   'accuracy_test': 0.4642857142857143,
   'correlation coefficient': 0.4968083495863984,
   'accuracy_cross_val': 0.6284271284271284,
   'estimator': KNeighborsClassifier(),
   'parameter_search_dict': {'algorithm': 'auto', 'n_neighbors': 5}},
  {'estimator_type': 'SVC',
   'accuracy_test': 0.35714285714285715,
   'correlation coefficient': 0.5856949962673031,
   'accuracy_cross_val': 0.6298701298701298,
   'estimator': SVC(C=1000, gamma=0.01),
   'parameter_search_dict': {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}},
  {'estimator_type': 'SVC',
   'accuracy_test': 0.35714285714285715,
   'correlation coefficient': 0.5602374510443873,
   'accuracy_cross_val': 0.645021645021645,
   'estimator': SVC(C=10, kernel='linear'),
   'parameter_search_dict': {'C': 10, 'kernel': 'linear'}},
  {'estimator_type': 'DecisionTreeClassifier',
   'accuracy_test': 0.5,
   'correlation coefficient': 0.5668447304063344,
   'accuracy_cros

In [27]:
result = {}
for key, val in res_classifier.items():
    df = pd.DataFrame(val)
    df["Metric"] = key
    result[key]= df

In [28]:
data = pd.concat(result)


In [29]:
data

accuracy_cross_val  accuracy_test  \
Dexterity                      0            0.491342       0.714286   
                               1            0.582973       0.678571   
                               2            0.583694       0.714286   
                               3            0.585859       0.642857   
                               4            0.599567       0.607143   
Flow of procedure              0            0.768398       0.892857   
                               1            0.753247       0.821429   
                               2            0.753247       0.821429   
                               3            0.815296       0.750000   
                               4            0.815296       0.750000   
Hand-eye Coordination          0            0.628427       0.464286   
                               1            0.629870       0.357143   
                               2            0.645022       0.357143   
                               3            0.582973       0.500000   
                               4            0.645022       0.500000   
Instrument-Tissue Manipulation 0            0.414863       0.500000   
                               1            0.523088       0.571429   
                               2            0.523088       0.571429   
                               3            0.491342       0.535714   
                               4            0.598846       0.607143   
Overall                        0            0.293651       0.071429   
                               1            0.278499       0.178571   
                               2            0.292929       0.107143   
                               3            0.293651       0.142857   
                               4            0.370851       0.142857   

                                  correlation coefficient  \
Dexterity                      0                 0.409186   
                               1                 0.335109   
                               2                 0.279161   
                               3                 0.530671   
                               4                 0.418335   
Flow of procedure              0                 0.509423   
                               1                 0.325779   
                               2                 0.428386   
                               3                 0.196574   
                               4                 0.328606   
Hand-eye Coordination          0                 0.496808   
                               1                 0.585695   
                               2                 0.560237   
                               3                 0.566845   
                               4                 0.476806   
Instrument-Tissue Manipulation 0                 0.201931   
                               1                      NaN   
                               2                      NaN   
                               3                 0.291491   
                               4                 0.289091   
Overall                        0                 0.523950   
                               1                 0.637012   
                               2                 0.515827   
                               3                 0.335271   
                               4                 0.500394   

                                                                          estimator  \
Dexterity                      0                             KNeighborsClassifier()   
                               1                                  SVC(C=1, gamma=1)   
                               2                          SVC(C=1, kernel='linear')   
                               3  DecisionTreeClassifier(max_depth=5, splitter='...   
                               4  (DecisionTreeClassifier(max_depth=6, max_featu...   
Flow of procedure              0                             KNeighborsClassif

In [30]:
res = {}
for p in preds:
    res[p]=trainer.run(regressors,params_regressors, names,p,method="regression")

Hand-eye Coordination KNeighborsRegressor()
Hand-eye Coordination SVR()
Hand-eye Coordination SVR()
Hand-eye Coordination DecisionTreeRegressor()
Hand-eye Coordination RandomForestRegressor()
Instrument-Tissue Manipulation KNeighborsRegressor()
Instrument-Tissue Manipulation SVR()
Instrument-Tissue Manipulation SVR()
Instrument-Tissue Manipulation DecisionTreeRegressor()
Instrument-Tissue Manipulation RandomForestRegressor()
Dexterity KNeighborsRegressor()
Dexterity SVR()
Dexterity SVR()
Dexterity DecisionTreeRegressor()
Dexterity RandomForestRegressor()
Flow of procedure KNeighborsRegressor()
Flow of procedure SVR()
Flow of procedure SVR()
Flow of procedure DecisionTreeRegressor()
Flow of procedure RandomForestRegressor()
Overall KNeighborsRegressor()
Overall SVR()
Overall SVR()
Overall DecisionTreeRegressor()
Overall RandomForestRegressor()


In [31]:
res

{'Hand-eye Coordination': [{'correlation coeff.': 0.3629582889469306,
   'estimator_type': 'KNeighborsRegressor',
   'mse_test': 0.5555555555555556,
   'mse_cross_val': 0.5816899150232485,
   'estimator': KNeighborsRegressor(n_neighbors=3),
   'parameter_search_dict': {'algorithm': 'auto', 'n_neighbors': 3}},
  {'correlation coeff.': -0.0548142897306225,
   'estimator_type': 'SVR',
   'mse_test': 1.847149514752852,
   'mse_cross_val': 0.9417567765841492,
   'estimator': SVR(C=1000, gamma=1),
   'parameter_search_dict': {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}},
  {'correlation coeff.': -0.5707940843908134,
   'estimator_type': 'SVR',
   'mse_test': 0.7021240415620476,
   'mse_cross_val': 0.753519048034604,
   'estimator': SVR(C=0.025, kernel='linear'),
   'parameter_search_dict': {'C': 0.025, 'kernel': 'linear'}},
  {'correlation coeff.': 0.40263822144831257,
   'estimator_type': 'DecisionTreeRegressor',
   'mse_test': 0.6924371282479391,
   'mse_cross_val': 0.9166067999952299,
   'est

In [32]:
result = {}
for key, val in res.items():
    df = pd.DataFrame(val)
    df["Metric"] = key
    result[key]= df

In [33]:
data_reg = pd.concat(result)


In [34]:
data_reg

correlation coeff.  \
Dexterity                      0            0.489805   
                               1            0.130772   
                               2            0.200448   
                               3            0.358298   
                               4            0.492692   
Flow of procedure              0            0.334472   
                               1           -0.086674   
                               2            0.183635   
                               3            0.296481   
                               4            0.357777   
Hand-eye Coordination          0            0.362958   
                               1           -0.054814   
                               2           -0.570794   
                               3            0.402638   
                               4            0.436249   
Instrument-Tissue Manipulation 0           -0.069324   
                               1           -0.126895   
                               2            0.132655   
                               3            0.250958   
                               4            0.196350   
Overall                        0            0.368687   
                               1            0.139345   
                               2            0.327879   
                               3            0.385943   
                               4            0.389747   

                                                                          estimator  \
Dexterity                      0                 KNeighborsRegressor(n_neighbors=3)   
                               1                               SVR(C=1000, gamma=1)   
                               2                      SVR(C=0.025, kernel='linear')   
                               3  DecisionTreeRegressor(max_depth=6, splitter='r...   
                               4  (DecisionTreeRegressor(max_depth=4, max_featur...   
Flow of procedure              0                              KNeighborsRegressor()   
                               1                               SVR(C=1000, gamma=1)   
                               2                      SVR(C=0.025, kernel='linear')   
                               3  DecisionTreeRegressor(max_depth=5, splitter='r...   
                               4  (DecisionTreeRegressor(max_depth=7, max_featur...   
Hand-eye Coordination          0                 KNeighborsRegressor(n_neighbors=3)   
                               1                               SVR(C=1000, gamma=1)   
                               2                      SVR(C=0.025, kernel='linear')   
                               3                 DecisionTreeRegressor(max_depth=5)   
                               4  (DecisionTreeRegressor(max_depth=6, max_featur...   
Instrument-Tissue Manipulation 0                 KNeighborsRegressor(n_neighbors=3)   
                               1                               SVR(C=1000, gamma=1)   
                               2                       SVR(C=1000, kernel='linear')   
                               3  DecisionTreeRegressor(max_depth=6, splitter='r...   
                               4  (DecisionTreeRegressor(max_depth=8, max_featur...   
Overall                        0                              KNeighborsRegressor()   
                               1                               SVR(C=1000, gamma=1)   
                               2                       SVR(C=1000, kernel='linear')   
                               3  DecisionTreeRegressor(max_depth=7, splitter='r...   
                               4  (DecisionTreeRegressor(max_depth=7, max_featur...   

                                         estimator_type  mse_cross_val  \
Dexterity                      0    KNeighborsRegressor       0.507937   
                               1                    SVR       2.212715   
                               2                    SVR       0.728849   
                  

In [35]:
#Fitting Linear models and correlation evaluation

In [36]:
from sklearn.metrics import make_scorer

In [37]:
from sklearn.neural_network import MLPRegressor
MLPRegressor(random_state=1, max_iter=500)

MLPRegressor(max_iter=500, random_state=1)

In [52]:
results = {}
for prediction_label in prediction_labels: 
    Y = JOINT[[prediction_label]]
    res = {}
    
#     model = LinearRegression()
    model = MLPClassifier(alpha=0.0001, hidden_layer_sizes=(20), max_iter=500, random_state=random.randint(0, 500), solver='adam')
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=30, random_state=random.randint(0, 500)) #70 | 30
    parameters = {"fit_intercept":[True, False], "normalize":[True, False]}
#     clf = GridSearchCV(model, parameters,scoring=make_scorer(mean_squared_error), cv=5) #
#     clf.fit(X, Y)
    model.fit(X_train,y_train.values.ravel())
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_pred,y_test.values.ravel()) #ypred vs gt y for test data
    
    y_pred_train = model.predict(X_train)
    mse_train = mean_squared_error(y_pred_train,y_train.values.ravel()) #ypred vs gt y for test data

                    
    test = cross_val_predict(model, X, Y.values.ravel(), cv=4)
    r = pearsonr(test, Y.values.ravel())
    
    
    
    print(clf.scoring)
#     res["scores"] = clf.best_score_
#     res["estimator"] = clf.best_estimator_
#     res["parameter_search_dict"] = clf.best_params_

    res["test"] = test
    res["Y"] = Y.values.ravel()
    res["corr."] = r[0]
    res["mse"] = mse
    res["mse_train"] = mse_train
    res["model"] = model
    
#     print(clf)
    
    results[prediction_label] = res

/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations 

make_scorer(mean_squared_error)


/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimiza

make_scorer(mean_squared_error)


/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimiza

...

/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimiza

make_scorer(mean_squared_error)


/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimiza

make_scorer(mean_squared_error)


/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [53]:
results

{'Hand-eye Coordination': {'test': array([4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3,
         4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 4, 4, 3,
         3, 3, 3, 1, 4, 4, 4, 4, 3, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4,
         3, 4, 3, 3, 3, 3, 3, 1, 3, 3, 1, 3, 3, 2, 3, 3, 1, 1, 4, 3, 4, 4,
         3, 3, 4, 4, 4]),
  'Y': array([4, 4, 4, 3, 3, 4, 3, 3, 3, 4, 3, 4, 4, 4, 3, 4, 4, 4, 4, 4, 3, 3,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 3, 3, 3, 3, 3, 4,
         4, 4, 4, 3, 4, 3, 4, 3, 3, 4, 3, 4, 3, 3, 4, 4, 4, 3, 4, 3, 3, 4,
         3, 4, 3, 3, 4, 3, 2, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 3, 3, 3, 3,
         4, 3, 3, 3, 3]),
  'corr.': 0.5388923637508254,
  'mse': 0.4666666666666667,
  'mse_train': 0.31746031746031744,
  'model': MLPClassifier(hidden_layer_sizes=20, max_iter=500, random_state=111)},
 'Instrument-Tissue Manipulation': {'test': array([3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 

In [378]:
results #by default R2 scores

{'Hand-eye Coordination': {'scores': 4.246726059640628,
  'estimator': LinearRegression(fit_intercept=False, normalize=True),
  'parameter_search_dict': {'fit_intercept': False, 'normalize': True}},
 'Instrument-Tissue Manipulation': {'scores': 3.1033817611163044,
  'estimator': LinearRegression(fit_intercept=False, normalize=True),
  'parameter_search_dict': {'fit_intercept': False, 'normalize': True}},
 'Dexterity': {'scores': 4.584124339603232,
  'estimator': LinearRegression(fit_intercept=False, normalize=True),
  'parameter_search_dict': {'fit_intercept': False, 'normalize': True}},
 'Flow of procedure': {'scores': 4.02030061656315,
  'estimator': LinearRegression(fit_intercept=False, normalize=True),
  'parameter_search_dict': {'fit_intercept': False, 'normalize': True}},
 'Effectualness': {'scores': 3.816783608032348,
  'estimator': LinearRegression(fit_intercept=False, normalize=True),
  'parameter_search_dict': {'fit_intercept': False, 'normalize': True}},
 'Overall': {'scores

In [379]:
from scipy.stats import pearsonr

results_corr = {}
for prediction_label in prediction_labels: 
    Y = JOINT[[prediction_label]]
    
    model = LinearRegression(fit_intercept=True, normalize=True)
  
    y_pred = cross_val_predict(model, X, Y, cv=10)  
    test = y_pred.ravel()
    test = [round(x) for x in test]
    
    

    r = pearsonr(test, Y.values.ravel())
    print(r)
    
    results_corr[prediction_label] = r[0]

(0.538122181507638, 2.6522431278273245e-08)
(0.14900979293442113, 0.15400247778009918)
(0.48546768369206683, 8.129939364459582e-07)
(0.354335143417622, 0.0004927824007958252)
(nan, 1.0)
(0.46992552437911983, 2.0104375117339473e-06)


/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [380]:
results_corr

{'Hand-eye Coordination': 0.538122181507638,
 'Instrument-Tissue Manipulation': 0.14900979293442113,
 'Dexterity': 0.48546768369206683,
 'Flow of procedure': 0.354335143417622,
 'Effectualness': nan,
 'Overall': 0.46992552437911983}

In [390]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

results = {}
for prediction_label in prediction_labels: 
    Y = JOINT[[prediction_label]]
    res = {}
    
    model = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=30, random_state=random.randint(0, 500)) #70 | 30
    parameters = {"fit_intercept":[True, False], "normalize":[True, False]}
    degree= 5
    polyreg=make_pipeline(PolynomialFeatures(degree),LinearRegression())
    y_pred = cross_val_predict(polyreg, X, Y, cv=10)  
    test = y_pred.ravel()
    test = [round(x) for x in test]
    
    r = pearsonr(test, Y.values.ravel())
    print(r)

#     clf = GridSearchCV(polyreg, parameters,scoring=make_scorer(mean_squared_error), cv=5) #
#     clf.fit(X, Y)
    
#     print(clf.scoring)
#     res["scores"] = clf.best_score_
#     res["estimator"] = clf.best_estimator_
#     res["parameter_search_dict"] = clf.best_params_
#     print(clf)
    
    results[prediction_label] = r


(-0.3598583317304763, 0.0003957809360452045)
(-0.1494312202636458, 0.15282812772500126)
(0.09546698822291205, 0.36267480567027777)
(0.11907830849112613, 0.255593082803883)


/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


(nan, 1.0)
(0.20154640207727284, 0.052710424212257675)


In [391]:
results

{'Hand-eye Coordination': (-0.3598583317304763, 0.0003957809360452045),
 'Instrument-Tissue Manipulation': (-0.1494312202636458, 0.15282812772500126),
 'Dexterity': (0.09546698822291205, 0.36267480567027777),
 'Flow of procedure': (0.11907830849112613, 0.255593082803883),
 'Effectualness': (nan, 1.0),
 'Overall': (0.20154640207727284, 0.052710424212257675)}

In [392]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [393]:
results = {}
for prediction_label in prediction_labels: 
    Y = JOINT[[prediction_label]]
    res = {}
    
    model = LogisticRegression()
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=30, random_state=random.randint(0, 500)) #70 | 30
    parameters = {}
    
    clf = GridSearchCV(model, parameters,scoring=make_scorer(mean_squared_error), cv=5) #
    clf.fit(X,Y)
#     clf = LogisticRegressionCV(cv=5, random_state=0).fit(X, Y)
    
    res["scores"] = clf.best_score_
    res["estimator"] = clf.best_estimator_
    res["parameter_search_dict"] = clf.best_params_
    
    results[prediction_label] = res

/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

In [394]:
results

{'Hand-eye Coordination': {'scores': 0.7456140350877193,
  'estimator': LogisticRegression(),
  'parameter_search_dict': {}},
 'Instrument-Tissue Manipulation': {'scores': 0.5678362573099415,
  'estimator': LogisticRegression(),
  'parameter_search_dict': {}},
 'Dexterity': {'scores': 0.6017543859649123,
  'estimator': LogisticRegression(),
  'parameter_search_dict': {}},
 'Flow of procedure': {'scores': 0.3333333333333333,
  'estimator': LogisticRegression(),
  'parameter_search_dict': {}},
 'Effectualness': {'scores': nan,
  'estimator': LogisticRegression(),
  'parameter_search_dict': {}},
 'Overall': {'scores': 13.121052631578948,
  'estimator': LogisticRegression(),
  'parameter_search_dict': {}}}

In [395]:
gt_y_test = y_test.values.ravel()
test = clf.predict(X_test)
test = test.ravel()
test = [int(round(x)) for x in test]
res = {}
mse = r2_score(test,gt_y_test)
res["accuracy"] = mse


In [396]:
res

{'accuracy': -2.2252666806107513}

In [397]:
pprint.pprint(list(zip(test,gt_y_test)))

[(19, 13),
 (19, 19),
 (19, 19),
 (10, 8),
 (17, 8),
 (19, 18),
 (17, 20),
 (19, 17),
 (19, 20),
 (16, 16),
 (17, 20),
 (19, 18),
 (19, 18),
 (18, 15),
 (19, 19),
 (17, 5),
 (19, 19),
 (19, 19),
 (18, 18),
 (10, 10),
 (17, 17),
 (19, 19),
 (19, 19),
 (18, 14),
 (19, 18),
 (20, 8),
 (20, 20),
 (17, 17),
 (17, 19),
 (18, 11)]


In [398]:
#Regression Equation

In [399]:
model.fit(X_train,y_train)

/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression()

In [400]:
model.coef_

array([[ 0.08620677, -0.07119962, -0.0554341 , -0.09525216,  0.21282421,
        -0.0148105 ,  0.        , -0.1875949 ,  0.04371481,  0.1536999 ],
       [-0.50838465, -0.1875491 , -0.02284558,  0.14477382,  0.61474801,
        -0.19281002,  0.        , -0.46896562,  0.17477064,  1.14484353],
       [ 0.17438953,  0.56454665, -0.03021606, -0.05901365,  0.71899137,
         0.12590296,  0.        , -0.4763595 ,  0.29886453,  0.22134019],
       [ 0.06025743, -0.18981416, -0.11410917, -0.21828115,  0.34661343,
        -0.07827934,  0.        , -0.27869889,  0.15424895,  0.26941828],
       [ 0.44862636,  0.31228892,  0.11006582, -0.07663794, -0.07638698,
         0.49180377,  0.        ,  0.55156841,  0.19684195, -0.38544598],
       [-0.12944354,  0.25113762, -0.01888128,  1.03433577, -0.1330939 ,
         0.22437987,  0.        ,  0.39323834,  0.06376914,  0.25705877],
       [-0.19906454,  0.46736339,  0.16660908,  0.06958207, -0.3978611 ,
         0.06439499,  0.        ,  0.76740771

In [401]:
model.intercept_

array([-1.64168699, -0.76120327, -0.8407757 , -0.58590744, -0.10773716,
       -0.05934975,  0.98735226,  0.68857671,  0.8910098 ,  1.42972156])

In [402]:
equation = str(model.intercept_[0])
for i in range(len(observations)):
    equation = equation + " + " + observations[i] + " * ("+str(model.coef_[0][i])+")" 
    
    

In [403]:
print(equation)

-1.6416869918667196 + NUMBER_OF_HITS * (0.0862067692191957) + HIT_INTENSITY * (-0.07119962049928276) + NUMBER_OF_TUG * (-0.05543410286195749) + TOTAL_FRAMES_IN_TUG * (-0.0952521572708486) + TIME * (0.21282421367578896) + NUMBER_OF_JERK_FRAMES * (-0.014810499646743691) + NUMBER_OF_RINGDROP * (0.0) + VARIANCE * (-0.18759490055732472) + ARC_LENGTH * (0.043714807622631795) + CURVATURE_MAX_COUNT * (0.15369989687612914)


In [404]:
model = LinearRegression(fit_intercept=True,normalize=True)

In [405]:
clf = GridSearchCV(model, parameters, scoring='r2', cv=5)

In [406]:
clf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LinearRegression(normalize=True), param_grid={},
             scoring='r2')

In [407]:
gt_y_test = y_test.values.ravel()
test = clf.predict(X_test)
test = test.ravel()
test = [round(x) for x in test]
res = {}
mse = mean_squared_error(test,gt_y_test)
res["accuracy"] = mse
res["scores"] = clf.best_score_
res["estimator"] = clf.best_estimator_
res["parameter_search_dict"] = clf.best_params_

In [408]:
res

{'accuracy': 9.9,
 'scores': -0.5054764899294202,
 'estimator': LinearRegression(normalize=True),
 'parameter_search_dict': {}}

In [409]:
# pprint.pprint(list(zip(test,gt_y_test)))

In [161]:
# pprint.pprint(res) #classification accurac score

{'Dexterity': [{'accuracy': 0.7727272727272727,
                'estimator': KNeighborsClassifier(n_neighbors=3),
                'parameter_search_dict': {'algorithm': 'auto',
                                          'n_neighbors': 3},
                'scores': 0.7806372549019608},
               {'accuracy': 0.6818181818181818,
                'estimator': SVC(C=100, gamma=0.01),
                'parameter_search_dict': {'C': 100,
                                          'gamma': 0.01,
                                          'kernel': 'rbf'},
                'scores': 0.7622549019607844},
               {'accuracy': 0.6363636363636364,
                'estimator': SVC(C=10, kernel='linear'),
                'parameter_search_dict': {'C': 10, 'kernel': 'linear'},
                'scores': 0.738970588235294},
               {'accuracy': 0.6818181818181818,
                'estimator': DecisionTreeClassifier(max_depth=6, splitter='random'),
                'parameter_search_dict': {

In [162]:
#REGRESSION

In [163]:
# res_regression = {}
# for p in preds:
#     res_regression[p]=trainer.run(regressors,params_regressors, names,p,method="regression")

Hand-eye Coordination KNeighborsRegressor()
Hand-eye Coordination SVR()


/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

Hand-eye Coordination SVR()


/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

...

Overall DecisionTreeRegressor()
Overall RandomForestRegressor()


/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn

/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn

/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn

/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/kunaldargan/ROBO_SURGERY/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [164]:
# pprint.pprint(res_regression) #regression mse

{'Dexterity': [{'accuracy': 0.18181818181818182,
                'estimator': KNeighborsRegressor(),
                'parameter_search_dict': {'algorithm': 'auto',
                                          'n_neighbors': 5},
                'scores': -0.1035555555555554},
               {'accuracy': 0.22727272727272727,
                'estimator': SVR(C=1, gamma=1),
                'parameter_search_dict': {'C': 1, 'gamma': 1, 'kernel': 'rbf'},
                'scores': 0.0716966167900606},
               {'accuracy': 0.3181818181818182,
                'estimator': SVR(C=0.25, kernel='linear'),
                'parameter_search_dict': {'C': 0.25, 'kernel': 'linear'},
                'scores': -0.06977869691045897},
               {'accuracy': 0.18181818181818182,
                'estimator': DecisionTreeRegressor(max_depth=4, splitter='random'),
                'parameter_search_dict': {'max_depth': 4, 'splitter': 'random'},
                'scores': 0.0058510802469135354},
         

In [173]:
# svr = SVR(gamma=2, C=1, max_iter=5000)
# X = JOINT[['NUMBER_OF_HITS', 'HIT_INTENSITY', 'NUMBER_OF_TUG',
#                   'TOTAL_FRAMES_IN_TUG', 'TIME','NUMBER_OF_JERK_FRAMES', 'NUMBER_OF_RINGDROP',
#                    'VARIANCE', 'ARC_LENGTH', 'CURVATURE_MAX_COUNT']]

# Y = JOINT[["Dexterity"]]
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=14, random_state=random.randint(0, 500)) #70 | 30


In [177]:
# scores = cross_val_score(SVR, X_train, y_train, scoring="mean_squared_error", cv=5)
svr.fit(X_train, y_train)

/home/kunaldargan/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=2,
  kernel='rbf', max_iter=5000, shrinking=True, tol=0.001, verbose=False)

In [178]:
test = svr.predict(X_test)

In [179]:
test

array([4.08196485, 3.26096152, 3.58883397, 3.98678664, 3.73804318,
       3.22488409, 3.94323538, 3.61045514, 3.58629549, 3.81229842,
       3.97074893, 4.01773206, 3.93062782, 3.9974055 ])

In [180]:
test = [round(x) for x in test]


In [181]:
gt_y_test = y_test[["Dexterity"]].values

In [182]:
gt_y_test = gt_y_test.ravel()

In [33]:
#Label ranking, External dataset with respect to medical

In [ ]:
#feature importance, dataset analysis, is it really valid, basis of selection criteria - inclusion and exclusion : Data bias

In [ ]:
#Validation and Abalation studies

In [1]:
# import sklearn and numpy libraries 
import numpy as np 
from sklearn.metrics import label_ranking_loss 
  
# take sample dataset 
y_true = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) 
y_pred_score = np.array([[0.75, 0.5, 1], [1, 0.2, 0.1], [0.1, 1, 0.9]]) 
  
# calculate and print label ranking loss 
print(label_ranking_loss(y_true, y_pred_score )) 
  
# this will give minimum ranking loss 
y_pred_score = np.array([[0.75, 0.5, 0.1], [0.1, 0.6, 0.1], [0.3, 0.3, 0.4]]) 
print(label_ranking_loss(y_true, y_pred_score ))

0.5
0.0


In [2]:
# take sample dataset 
y_true = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) 
y_pred_score = np.array([[0.5, 0.75, 1], [1, 0.2, 0.1], [0.1, 1, 0.9]]) 
  
# calculate and print label ranking loss 
print(label_ranking_loss(y_true, y_pred_score )) #2/3

0.6666666666666666


In [ ]:
# Summation of first 4 of expert 